### use llm for topic representation

In [43]:
import os,sys
sys.path.insert(0,'../libs')
from utils import load_json
import openai
from llm_utils import BSAgent_legacy
import numpy as np
from bertopic import BERTopic
import pandas as pd
import re
import json
## load API Key
key = load_json('/home/chuang/Dev/Keys/openai_key.json') 
os.environ['OPENAI_API_KEY'] = key['ChatGPT']['API_KEY']
openai.api_key  = os.getenv('OPENAI_API_KEY')

- load legacy llm agent class, as fund server having problem with the new version

In [4]:
llm_agent  = BSAgent_legacy(model="gpt-3.5-turbo-1106", 
                    temperature=0)

In [5]:
## just run one test, make sure the api works 
pt = {'System':'You are a helpful assistant.',
      'Human':'What is your name?'}
res = llm_agent.get_response_content(prompt_template=pt)
print(res)  


I am a language model AI created by OpenAI, and I don't have a personal name. You can just call me "Assistant." How can I assist you today?


In [7]:
res_folder = '/data/chuang/news_scrape/data/news_search_res'
emb_path  = os.path.join(res_folder,'sentence_embeddings.npy')
docs_path = os.path.join(res_folder,'docs.npy')
topic_model_out_path = os.path.join(res_folder,'topic_model')

In [12]:
topic_model = BERTopic.load(topic_model_out_path)
topic_model.calculate_probabilities = False ## do not calculate probs for all topics 
## load training doc and embeddings 
embeddings = np.load(emb_path)
docs = np.load(docs_path)

In [21]:
topic_model.visualize_barchart()

In [46]:
delim = '\n----------------\n----------------\n'

topic_df = topic_model.get_topic_info()
topic_df['topic_words'] = topic_model.generate_topic_labels(nr_words=10,
                                                            topic_prefix=False,
                                                            word_length=50,
                                                            separator=", ")
rep_doc_dict = topic_model.get_representative_docs()      
rep_doc_dict = {k:delim.join([re.sub(r'\n+', '\n', text) for text in v]) for k,v in rep_doc_dict.items()} 
topic_df.head()

,Topic,Count,Name,topic_words
0,-1,590,-1_ndp_minister_opposition_prime minister,"ndp, minister, opposition, prime minister, tea..."
1,0,486,0_caribbean_st vincent_vincent grenadines_vinc...,"caribbean, st vincent, vincent grenadines, vin..."
2,1,192,1_party ndp_ndp_democratic party_new democratic,"party ndp, ndp, democratic party, new democrat..."
3,2,113,2_sentencing_convicted_accused_prosecution,"sentencing, convicted, accused, prosecution, p..."
4,3,112,3_west indies_indies_batsman_bowler,"west indies, indies, batsman, bowler, cricket,..."


In [103]:
llm_agent.get_response_content(prompt_template=pt,conv_history=[])

"The article discusses the predictions and statements made by members of the Unity Labour Party (ULP) and the New Democratic Party (NDP) in St. Vincent and the Grenadines regarding the upcoming general elections. ULP's Luke Browne predicts a landslide victory for his party, while NDP's Linton Lewis criticizes members who joined the party because of the leader. Additionally, NDP's Arnhim Eustace expresses confidence in winning the next general elections and pledges a government of inclusiveness. The article also highlights the economic and political challenges faced by both parties."

In [54]:
for k,v in rep_doc_dict.items():
    

In [13]:
t,p = topic_model.transform(docs)

Batches:   0%|          | 0/93 [00:00<?, ?it/s]

In [147]:
summary_pt = {'System':"You're a helpful assistant. Your task is to analyse hotel topics in news articles.",
      'Human':"""
Please summarize the following new article with less then 100 words: 
----------------
----------------

{DOCUMENTS}

----------------
----------------
Please write your summery below:
"""}

topic_pt = {'System':"You're a helpful assistant. Your task is to analyse hotel topics in news articles.",
      'Human':"""
I have a topic that contains the following documents delimited with ----------------: 
----------------
----------------

{DOCUMENTS}

----------------
----------------

The topic is described by the following keywords: {KEYWORDS}

Based on the information above, extract a short topic label, a list of topic words and a short descrption of the topic.
Make sure the label has no more than 3 words.
Make sure topic words arel less than 10 words.
And also makse sure the description is only a short highlevel sumamry.  
Please respond in clean json format as follow:
topic: <topic label>,
topic_words: [<topic workds>],
description: <topic description>

"""}
topic_pt['Human'] = topic_pt['Human'].format(DOCUMENTS=rep_doc_dict[1],KEYWORDS=topic_df.iloc[1]['topic_words'])

In [149]:
res = llm_agent.get_response_content(prompt_template=topic_pt,conv_history=[])
print(json.loads(llm_agent.parse_load_json_str(res)))  

{'topic': 'St Vincent Politics', 'topic_words': ['St Vincent', 'Vincent Grenadines', 'Vincentians', 'Elections', 'Opposition Party', 'Government', 'Leader', 'Politics'], 'description': 'Analysis of political developments and party dynamics in St Vincent and the Grenadines, including discussions on upcoming elections, party leadership, and government policies.'}
